In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from fastDamerauLevenshtein import damerauLevenshtein
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from tensorflow.keras import layers

In [3]:
df = pd.read_csv('data/single_step_df_ints_2022-03-15.csv')

In [4]:
df

,input,target,start_coords.x,start_coords.y,start_coords.z,row,l.already_seen,d.already_seen,g.already_seen,f.already_seen,...,h.food_k,h.strong_k,h.mid_k,coordinates_q.x,coordinates_q.y,coordinates_q.z,q.containment,q.food_k,q.strong_k,q.mid_k
0,<start>,p,-0.451354,-0.413918,0.156247,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,p,o,0.513000,-0.531000,0.740000,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,o,c,0.513000,-0.531000,0.740000,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,c,g,0.513000,-0.531000,0.740000,0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,g,k,0.513000,-0.531000,0.740000,0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,h,h,0.000000,4.000000,1.000000,189,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1458,h,s,0.000000,4.000000,1.000000,189,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1459,s,d,0.000000,2.000000,2.000000,189,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,d,g,0.000000,1.000000,3.000000,189,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
float_cols = df.select_dtypes(include=['float64']).columns
df.loc[:, float_cols] = df.loc[:, float_cols].fillna(-99)

In [6]:
df['target'].unique()
df['target'].value_counts()

df['target'].loc[df['target'] == 'l']

727    l
Name: target, dtype: object

In [7]:
nr_of_sequences = len(df[df['input'] == '<start>'])

In [8]:
# remove row with target that only occurs once

df_new = df.drop(index=727, axis=0)
df_new.reset_index(inplace=True, drop=True)

df_new

,input,target,start_coords.x,start_coords.y,start_coords.z,row,l.already_seen,d.already_seen,g.already_seen,f.already_seen,...,h.food_k,h.strong_k,h.mid_k,coordinates_q.x,coordinates_q.y,coordinates_q.z,q.containment,q.food_k,q.strong_k,q.mid_k
0,<start>,p,-0.451354,-0.413918,0.156247,0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1,p,o,0.513000,-0.531000,0.740000,0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
2,o,c,0.513000,-0.531000,0.740000,0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
3,c,g,0.513000,-0.531000,0.740000,0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
4,g,k,0.513000,-0.531000,0.740000,0,0.0,0.0,1.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,h,h,0.000000,4.000000,1.000000,189,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1457,h,s,0.000000,4.000000,1.000000,189,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1458,s,d,0.000000,2.000000,2.000000,189,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1459,d,g,0.000000,1.000000,3.000000,189,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0


In [9]:
# generate stratified split for train - test/val

split = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)

for train_index, test_index in split.split(df_new, df_new['target']):
    strat_train = df_new.loc[train_index]
    strat_test_val = df_new.loc[test_index]

In [10]:
# split test_val into test and val

split_test_val = len(strat_test_val)//2

strat_test = strat_test_val[:split_test_val]
strat_val = strat_test_val[split_test_val:]

In [11]:
# generate list of labels to pass to MultiLabelBinarizer so there's the same number of
# classes for all datasets

labels = df_new['target'].unique()

In [12]:
def create_dataset(dataframe, labels, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels_ds = df.pop('target')
    encoder = MultiLabelBinarizer(classes=labels)
    encoded_labels = encoder.fit_transform(labels_ds)
    
    df = {key: value[:, tf.newaxis] for key, value in df.items()}
    dataset = tf.data.Dataset.from_tensor_slices((dict(df), encoded_labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(dataframe))
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(batch_size)
    
    return dataset

In [13]:
batch_size = 256

In [14]:
train_ds = create_dataset(strat_train, labels, batch_size=batch_size)
val_ds = create_dataset(strat_val, labels, shuffle=False, batch_size=batch_size)
test_ds = create_dataset(strat_test, labels, shuffle=False, batch_size=batch_size)

/tmp/ipykernel_71905/1510453973.py:7: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:, tf.newaxis] for key, value in df.items()}
2022-03-15 13:38:58.255927: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-15 13:38:58.256394: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-15 13:38:58.256728: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-03-15 13:38:58.256989: W tensorflow

In [15]:
[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of targets:', label_batch)

Every feature: ['input', 'start_coords.x', 'start_coords.y', 'start_coords.z', 'row', 'l.already_seen', 'd.already_seen', 'g.already_seen', 'f.already_seen', 'r.already_seen', 'z.already_seen', 'c.already_seen', 'w.already_seen', 'k.already_seen', 'p.already_seen', 'q.already_seen', 't.already_seen', 'h.already_seen', 'x.already_seen', 's.already_seen', 'e.already_seen', 'm.already_seen', 'b.already_seen', 'n.already_seen', 'o.already_seen', 'a.already_seen', 'coordinates_p.x', 'coordinates_p.y', 'coordinates_p.z', 'p.containment', 'p.food_k', 'p.strong_k', 'p.mid_k', 'coordinates_o.x', 'coordinates_o.y', 'coordinates_o.z', 'o.containment', 'o.food_k', 'o.strong_k', 'o.mid_k', 'coordinates_c.x', 'coordinates_c.y', 'coordinates_c.z', 'c.containment', 'c.food_k', 'c.strong_k', 'c.mid_k', 'coordinates_g.x', 'coordinates_g.y', 'coordinates_g.z', 'g.containment', 'g.food_k', 'g.strong_k', 'g.mid_k', 'coordinates_k.x', 'coordinates_k.y', 'coordinates_k.z', 'k.containment', 'k.food_k', 'k.str

In [16]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
    normalizer = layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
    normalizer.adapt(feature_ds)
    
    return normalizer

In [17]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    if dtype == 'string':
        index = layers.StringLookup(max_tokens=max_tokens)
    else:
        index = layers.IntegerLookup(max_tokens=max_tokens)
        
    # prepare tf.data.Dataset that only yields the feature    
    feature_ds = dataset.map(lambda x, y: x[name])
    
    # learn set of possible values and assign fixed int index
    index.adapt(feature_ds)
    
    # encode int indices
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())
    
    # apply multi-hot encoding to indices
    # lambda function captures the layer to include them in Keras functional models later
    return lambda feature: encoder(index(feature))

In [26]:
def create_input_data(dataframe):
    all_inputs = []
    encoded_features = []
    
    for header in dataframe.columns:
        # numerical features
        if 'coord' in header or 'already' in header:
            numeric_col = tf.keras.Input(shape=(1,), name=header)
            normalization_layer = get_normalization_layer(header, train_ds)
            encoded_numeric_col = normalization_layer(numeric_col)
            all_inputs.append(numeric_col)
            encoded_features.append(encoded_numeric_col)
        
        # categorical features
        elif 'containment' in header or 'food' in header or 'mid' in header or \
        'strong' in header:
            categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='int64')
            encoding_layer = get_category_encoding_layer(name=header,
                                                        dataset=train_ds,
                                                        dtype='int64')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
        elif header == 'input':
            categorical_col = tf.keras.Input(shape=(1,), name='input', dtype='string')
            encoding_layer = get_category_encoding_layer(name='input',
                                                        dataset=train_ds,
                                                        dtype='string')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
    return all_inputs, encoded_features

In [27]:
all_inputs, encoded_features = create_input_data(df)

In [28]:
# use model from tutorial with dense layers

all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(512, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(256, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.2)(x)
output = tf.keras.layers.Dense(20)(x)

model = tf.keras.Model(all_inputs, output)

In [29]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             metrics=["accuracy"])

In [81]:
#tf.keras.utils.plot_model(model, show_shapes=True, rankdir='LR')

In [30]:
model.fit(train_ds, epochs=100, validation_data=val_ds)

Epoch 1/100


/home/zauri/files/anaconda3/envs/tf2.7/lib/python3.9/site-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['row'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


4/4 [==============================] - 8s 923ms/step - loss: 3.0989 - accuracy: 0.0851 - val_loss: 2.5677 - val_accuracy: 0.1500
Epoch 2/100
4/4 [==============================] - 0s 43ms/step - loss: 2.4233 - accuracy: 0.1663 - val_loss: 2.2654 - val_accuracy: 0.1909
Epoch 3/100
4/4 [==============================] - 0s 42ms/step - loss: 2.1675 - accuracy: 0.2339 - val_loss: 2.1221 - val_accuracy: 0.3000
Epoch 4/100
4/4 [==============================] - 0s 47ms/step - loss: 2.0370 - accuracy: 0.2769 - val_loss: 2.0289 - val_accuracy: 0.3318
Epoch 5/100
4/4 [==============================] - 0s 43ms/step - loss: 1.9439 - accuracy: 0.2838 - val_loss: 1.9528 - val_accuracy: 0.3227
Epoch 6/100
4/4 [==============================] - 0s 43ms/step - loss: 1.8500 - accuracy: 0.3268 - val_loss: 1.8908 - val_accuracy: 0.3727
Epoch 7/100
4/4 [==============================] - 0s 44ms/step - loss: 1.7958 - accuracy: 0.3611 - val_loss: 1.8501 - val_accuracy: 0.3500
Epoch 8/100
4/4 [==============

4/4 [==============================] - 0s 45ms/step - loss: 0.8982 - accuracy: 0.6996 - val_loss: 1.2720 - val_accuracy: 0.5682
Epoch 60/100
4/4 [==============================] - 0s 43ms/step - loss: 0.8959 - accuracy: 0.6879 - val_loss: 1.2741 - val_accuracy: 0.5773
Epoch 61/100
4/4 [==============================] - 0s 45ms/step - loss: 0.8616 - accuracy: 0.7162 - val_loss: 1.2785 - val_accuracy: 0.5727
Epoch 62/100
4/4 [==============================] - 0s 47ms/step - loss: 0.8609 - accuracy: 0.7016 - val_loss: 1.2779 - val_accuracy: 0.5864
Epoch 63/100
4/4 [==============================] - 0s 45ms/step - loss: 0.8545 - accuracy: 0.6996 - val_loss: 1.2771 - val_accuracy: 0.5682
Epoch 64/100
4/4 [==============================] - 0s 45ms/step - loss: 0.8568 - accuracy: 0.6996 - val_loss: 1.2768 - val_accuracy: 0.5773
Epoch 65/100
4/4 [==============================] - 0s 44ms/step - loss: 0.8484 - accuracy: 0.7094 - val_loss: 1.2851 - val_accuracy: 0.5818
Epoch 66/100
4/4 [========

In [31]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

1/1 [==============================] - 0s 102ms/step - loss: 1.4455 - accuracy: 0.5160
Accuracy 0.5159817337989807


In [38]:
model.save('models/next_obj_classifier_2022-03-15')

2022-03-15 14:05:03.524192: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/next_obj_classifier_2022-03-15/assets


INFO:tensorflow:Assets written to: models/next_obj_classifier_2022-03-15/assets


In [86]:
reloaded_model = tf.keras.models.load_model('next_obj_classifier')

In [38]:
# test prediction for one sample

In [87]:
sample = df.loc[0].drop('target').to_dict()

In [88]:
input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}

In [89]:
prediction = reloaded_model.predict(input_dict)
prediction = tf.nn.sigmoid(prediction[0])

In [90]:
# get label for prediction

pred_label = labels[np.argmax(prediction)]
pred_label

'p'

In [32]:
def get_prequential_error(dataframe, model, labels, nr_of_sequences):
    errors = [[] for seq in range(0, nr_of_sequences + 1)]
    start_token_count = 0
    sequence_nr = 0
    
    for row in range(0, len(dataframe)): 
        observed_target = dataframe.loc[row, 'target']
        sample = dataframe.loc[row].drop('target').to_dict()
        input_dict = {name: tf.convert_to_tensor([value]) for name, value in 
                          sample.items()}
        predicted_target = model.predict(input_dict)
        predicted_target = tf.nn.sigmoid(predicted_target[0])
            
        pred_label = labels[np.argmax(predicted_target)]
        error = 1 - damerauLevenshtein(pred_label, observed_target)
        errors[sequence_nr].append(error)
        
        if row != 0 and dataframe.loc[row, 'input'] == '<start>':
            start_token_count += 1
        
        if start_token_count > 0:
            sequence_nr += 1
            start_token_count = 0
            
    return errors

In [33]:
errors = get_prequential_error(df, model, labels, nr_of_sequences)

In [34]:
summed_error = [sum(error) for error in errors[:-1]]

In [35]:
np.median(summed_error)

2.0

In [37]:
#with open('results/nn_spatialinfo_prequential_summed_2022-03-15.txt', 'w') as file:
#    file.write(str(summed_error))